In [2]:
import torch
import pandas as pd
import numpy as np
import os
import torch.utils
import torchvision.transforms
from torch import nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
%matplotlib inline

torch.set_printoptions(threshold=1000)

In [3]:
temp_pandas = pd.read_csv(os.path.join("../Cyclist_Detection/Dataset","src","src.csv"))

In [13]:
temp_pandas.iloc[4]

Unnamed: 0            4
Images:       000004.pt
Lables        000004.pt
Name: 4, dtype: object

In [ ]:
#Writing our Dataset class
class YoloPtDataset(torch.utils.data.Dataset):
    """Dataset with tensor format images and Yolo format labels"""

    def __init__(self,root_dir,transform=None):
        try:
            self.paths = pd.read_csv(os.path.join(root_dir,"src","src.csv"))
        except:
            raise ValueError("The directory doesn't have a src file with the paths to all the images")

        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
